In [1]:
from rdkit.Chem import rdFingerprintGenerator
from rdkit import Chem
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import os

In [2]:
SMILES = pd.read_csv("/home/acomajuncosa/Documents/chembl-antimicrobial-tasks/config/chembl_activities/compound_structures.csv")['canonical_smiles'].tolist()

In [3]:
len(SMILES)

2854815

In [4]:
def clip_sparse(vect, nBits=2048):
  l = [0] * nBits
  for i, v in vect.GetNonzeroElements().items():
    l[i] = v if v < np.iinfo(np.int8).max else np.iinfo(np.int8).max
  return l

In [5]:
def smiles_to_ecfp(smiles, radius=2, nBits=2048):
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
    OUTPUT_SMILES, X = [], []
    for smi in tqdm(smiles):
        try:
            mol = Chem.MolFromSmiles(smi)
            ecfp = mfpgen.GetCountFingerprint(mol)
            ecfp = clip_sparse(ecfp)
            X.append(ecfp)
            OUTPUT_SMILES.append(smi)
        except:
            pass  
    return OUTPUT_SMILES, np.array(X, dtype=np.int8)

In [8]:
OUTPUT_SMILES, X = smiles_to_ecfp(SMILES[:1000000])

100%|██████████| 1000000/1000000 [04:04<00:00, 4085.97it/s]


In [13]:
10_000_000_000 / 1_000_000

10000.0

In [ ]:
pickle.dump(X, open("../X.pkl", "wb"))

In [10]:
from scipy.sparse import csr_matrix, save_npz

def save_fingerprints(fingerprints, filename):
    """Save one fingerprint array efficiently"""
    sparse_fp = csr_matrix(fingerprints.astype(np.uint8))
    save_npz(filename, sparse_fp)

save_fingerprints(X, open("../X.npz", "wb"))

In [12]:
len(X) / 1000000

0.999998